# Holder Agent

In [58]:
import requests
import json
import qrcode
import uuid
import base64
import re
import jwt


In [59]:
# Set the URL for the API endpoint
# Need to specify the host as host.docker.internal as the Prism Agents and the Prism Playground are running in two isolated docker environments
base_url = "http://host.docker.internal:8090"

headers = {'Content-Type': 'application/json'}

## Connections

In [60]:
path = '/prism-agent/connections'

# Make the request
response = requests.get(base_url + path, headers=headers)

# Check the response status code
if response.status_code == 200:
    # Print the response content
    loaded_json = json.loads(response.content.decode())
    print(json.dumps(loaded_json, indent=2))
else:
    # Print an error message
    print("Error: The API returned a non-200 status code")

{
  "contents": [],
  "kind": "Collection",
  "self": "/collections"
}


## Fetch invitationUrl from Issuer Agent

In [61]:
%store -r invitationUrlbase64
print(invitationUrlbase64)

# Standard Base64 Decoding
decodedBytes = base64.b64decode(invitationUrlbase64)
decodedStr = str(decodedBytes, "utf-8")

print("\n", decodedStr)


eyJpZCI6IjhmMzM5YWFlLTVlNTktNGYzNi1hNzdlLTBmMzFhYjU4ZTZmNCIsInR5cGUiOiJodHRwczovL2RpZGNvbW0ub3JnL291dC1vZi1iYW5kLzIuMC9pbnZpdGF0aW9uIiwiZnJvbSI6ImRpZDpwZWVyOjIuRXo2TFNrREVSNFpIU2NBRERvV3NkQ0VwbnhUd05zYVVQaFlrMjk1aWRNTEJ5dnpGQy5WejZNa2Z5emtjaW1yZVo1RTVZQXMzWjZZNHZDMk1XWUdYOUVQV2Q5OWlRRkczQzNRLlNleUowSWpvaVpHMGlMQ0p6SWpvaWFIUjBjRG92TDJodmMzUXVaRzlqYTJWeUxtbHVkR1Z5Ym1Gc09qZ3dPREF2Wkdsa1kyOXRiU0lzSW5JaU9sdGRMQ0poSWpwYkltUnBaR052YlcwdmRqSWlYWDAiLCJib2R5Ijp7ImdvYWxfY29kZSI6ImNvbm5lY3QiLCJnb2FsIjoiRXN0YWJsaXNoIGEgdHJ1c3QgY29ubmVjdGlvbiBiZXR3ZWVuIHR3byBwZWVycyIsImFjY2VwdCI6W119fQ==

 {"id":"8f339aae-5e59-4f36-a77e-0f31ab58e6f4","type":"https://didcomm.org/out-of-band/2.0/invitation","from":"did:peer:2.Ez6LSkDER4ZHScADDoWsdCEpnxTwNsaUPhYk295idMLByvzFC.Vz6MkfyzkcimreZ5E5YAs3Z6Y4vC2MWYGX9EPWd99iQFG3C3Q.SeyJ0IjoiZG0iLCJzIjoiaHR0cDovL2hvc3QuZG9ja2VyLmludGVybmFsOjgwODAvZGlkY29tbSIsInIiOltdLCJhIjpbImRpZGNvbW0vdjIiXX0","body":{"goal_code":"connect","goal":"Establish a trust connection between two peer

## Accept an OOB connection invitation from Issuer Agent
> Replace `{RAW_INVITATION}` with the value of the '_oob' query string parameter from the invitation URL above
```bash
curl -X 'POST' \
	'http://localhost:8090/prism-agent/connection-invitations' \
	-H 'Content-Type: application/json' \
	-d '{
		"invitation": "{RAW_INVITATION}"
	}' | jq
```

In [62]:
path = '/prism-agent/connection-invitations'
data = {"invitation": invitationUrlbase64}
# Make the request
response = requests.post(base_url + path, headers=headers, json=data)

# Check the response status code
if response.status_code == 200:
    # Print the response content
    loaded_json = json.loads(response.content.decode())
    print(json.dumps(loaded_json, indent=2))
    invitation = json.loads(response.content.decode())
    invitationUrl = invitation['invitation']['invitationUrl']
    # invitationUrlbase64 = invitation['invitation']['invitationUrl'].split('=')[1]
    connection_id = invitation['connectionId']
else:
    # Print an error message
    print("Error: The API returned a non-200 status code")

{
  "connectionId": "49070030-7557-4e58-8545-20ae56d873e5",
  "createdAt": "2022-12-14T18:41:05Z",
  "invitation": {
    "from": "did:peer:2.Ez6LSkDER4ZHScADDoWsdCEpnxTwNsaUPhYk295idMLByvzFC.Vz6MkfyzkcimreZ5E5YAs3Z6Y4vC2MWYGX9EPWd99iQFG3C3Q.SeyJ0IjoiZG0iLCJzIjoiaHR0cDovL2hvc3QuZG9ja2VyLmludGVybmFsOjgwODAvZGlkY29tbSIsInIiOltdLCJhIjpbImRpZGNvbW0vdjIiXX0",
    "id": "8f339aae-5e59-4f36-a77e-0f31ab58e6f4",
    "invitationUrl": "https://domain.com/path?_oob=eyJpZCI6IjhmMzM5YWFlLTVlNTktNGYzNi1hNzdlLTBmMzFhYjU4ZTZmNCIsInR5cGUiOiJodHRwczovL2RpZGNvbW0ub3JnL291dC1vZi1iYW5kLzIuMC9pbnZpdGF0aW9uIiwiZnJvbSI6ImRpZDpwZWVyOjIuRXo2TFNrREVSNFpIU2NBRERvV3NkQ0VwbnhUd05zYVVQaFlrMjk1aWRNTEJ5dnpGQy5WejZNa2Z5emtjaW1yZVo1RTVZQXMzWjZZNHZDMk1XWUdYOUVQV2Q5OWlRRkczQzNRLlNleUowSWpvaVpHMGlMQ0p6SWpvaWFIUjBjRG92TDJodmMzUXVaRzlqYTJWeUxtbHVkR1Z5Ym1Gc09qZ3dPREF2Wkdsa1kyOXRiU0lzSW5JaU9sdGRMQ0poSWpwYkltUnBaR052YlcwdmRqSWlYWDAiLCJib2R5Ijp7ImdvYWxfY29kZSI6ImNvbm5lY3QiLCJnb2FsIjoiRXN0YWJsaXNoIGEgdHJ1c3QgY29ubmVjdGlvbiBiZXR3ZWV

## Retrieving the list of connections
```bash
curl -X 'GET' 'http://localhost:8090/prism-agent/connections' | jq
```

In [63]:

path = '/prism-agent//connections/' + connection_id

# Make the request
response = requests.get(base_url + path, headers=headers)

# Check the response status code
if response.status_code == 200:
    # Print the response content
    loaded_json = json.loads(response.content.decode())
    print(json.dumps(loaded_json, indent=2))
    STATE = loaded_json['state']
    
else:
    # Print an error message
    print("Error: The API returned a non-200 status code")

{
  "connectionId": "49070030-7557-4e58-8545-20ae56d873e5",
  "createdAt": "2022-12-14T18:41:05Z",
  "invitation": {
    "from": "did:peer:2.Ez6LSkDER4ZHScADDoWsdCEpnxTwNsaUPhYk295idMLByvzFC.Vz6MkfyzkcimreZ5E5YAs3Z6Y4vC2MWYGX9EPWd99iQFG3C3Q.SeyJ0IjoiZG0iLCJzIjoiaHR0cDovL2hvc3QuZG9ja2VyLmludGVybmFsOjgwODAvZGlkY29tbSIsInIiOltdLCJhIjpbImRpZGNvbW0vdjIiXX0",
    "id": "8f339aae-5e59-4f36-a77e-0f31ab58e6f4",
    "invitationUrl": "https://domain.com/path?_oob=eyJpZCI6IjhmMzM5YWFlLTVlNTktNGYzNi1hNzdlLTBmMzFhYjU4ZTZmNCIsInR5cGUiOiJodHRwczovL2RpZGNvbW0ub3JnL291dC1vZi1iYW5kLzIuMC9pbnZpdGF0aW9uIiwiZnJvbSI6ImRpZDpwZWVyOjIuRXo2TFNrREVSNFpIU2NBRERvV3NkQ0VwbnhUd05zYVVQaFlrMjk1aWRNTEJ5dnpGQy5WejZNa2Z5emtjaW1yZVo1RTVZQXMzWjZZNHZDMk1XWUdYOUVQV2Q5OWlRRkczQzNRLlNleUowSWpvaVpHMGlMQ0p6SWpvaWFIUjBjRG92TDJodmMzUXVaRzlqYTJWeUxtbHVkR1Z5Ym1Gc09qZ3dPREF2Wkdsa1kyOXRiU0lzSW5JaU9sdGRMQ0poSWpwYkltUnBaR052YlcwdmRqSWlYWDAiLCJib2R5Ijp7ImdvYWxfY29kZSI6ImNvbm5lY3QiLCJnb2FsIjoiRXN0YWJsaXNoIGEgdHJ1c3QgY29ubmVjdGlvbiBiZXR3ZWV

In [77]:
import time
from termcolor import colored,cprint

path = '/prism-agent//connections/' + connection_id

# print('Current state for ConnectionId {} is {}'.format(connection_id,STATE))
print(colored("Current state for ConnectionId {} is {}".format(connection_id,STATE), "magenta", attrs=["bold"]))
while STATE != 'ConnectionResponseReceived':
    # Make the request
    response = requests.get(base_url + path, headers=headers)

    # Check the response status code
    if response.status_code == 200:
        # Print the response content
        loaded_json = json.loads(response.content.decode())
        # print(json.dumps(loaded_json, indent=2))
        STATE = loaded_json['state']
        their_did = loaded_json['theirDid']
        print(colored("ConnectionId {0} is not in active state yet. State: {1}".format(connection_id,STATE), "yellow", attrs=["bold"]))

    else:
        # Print an error message
        print("Error: The API returned a non-200 status code")
        

    time.sleep(1)
    
# print('ConnectionId: {0} is now active. Continue with notebook'.format(connection_id))
print(colored("ConnectionId: {0} is now active. Continue with notebook".format(connection_id), "green", attrs=["bold"]))


Current state for ConnectionId aa7aaf4d-eeb8-437e-8318-19b0033d15fb is ConnectionResponseReceived
ConnectionId: aa7aaf4d-eeb8-437e-8318-19b0033d15fb is now active. Continue with notebook


## Receive Credentials from Issuer

### Accept credential from Issuer

### Retrieving the list of issue records
```bash
curl -X 'GET' 'http://localhost:8090/prism-agent/issue-credentials/records' | jq
```

In [65]:
path = '/prism-agent/issue-credentials/records'

# Make the request
response = requests.get(base_url + path, headers=headers)

# Check the response status code
if response.status_code == 200:
    # Print the response content
    loaded_json = json.loads(response.content.decode())
    print(json.dumps(loaded_json, indent=2))
    record_id = loaded_json['items'][0]['recordId']
else:
    # Print an error message
    print("Error: The API returned a non-200 status code")
    print(response.content)

{
  "count": 1,
  "items": [
    {
      "claims": {
        "birthdate": "01/01/2000",
        "firstname": "Alice",
        "lastname": "Wonderland"
      },
      "createdAt": "2022-12-14T18:42:13Z",
      "protocolState": "OfferReceived",
      "recordId": "08eb1750-a446-4b06-b89f-34c8cea31e64",
      "role": "Holder",
      "subjectId": "did:peer:2.Ez6LSdEYSAtkCArFgUXXY5g4RhXP23ej1Ynu8UejzjU9YrX3E.Vz6Mksj7wesBWcbKV7so6ipvaKYY9wse6aXc7rEYxWLXdJ8zw.SeyJ0IjoiZG0iLCJzIjoiaHR0cDovL2hvc3QuZG9ja2VyLmludGVybmFsOjgwOTAvZGlkY29tbSIsInIiOltdLCJhIjpbImRpZGNvbW0vdjIiXX0"
    }
  ],
  "limit": 0,
  "offset": 0
}


### Accepting the credential offer 

Replace `{RECORD_ID}` with the UUID of the record from the previous list
```bash
curl -X 'POST' 'http://localhost:8090/prism-agent/issue-credentials/records/{RECORD_ID}/accept-offer' | jq
```

In [66]:
%store -r credential_record_id
print("Issuer credential recordId:", credential_record_id)
print("Holder credential recordId:",record_id)

Issuer credential recordId: 51dc271c-4abf-472a-89b9-253d2256c61a
Holder credential recordId: 08eb1750-a446-4b06-b89f-34c8cea31e64


In [67]:
path = '/prism-agent/issue-credentials/records/' + record_id + '/accept-offer'

# Make the request
response = requests.post(base_url + path, headers=headers)

# Check the response status code
if response.status_code == 200:
    # Print the response content
    loaded_json = json.loads(response.content.decode())
    print(json.dumps(loaded_json, indent=2))
    
else:
    # Print an error message
    print("Error: The API returned a non-200 status code")
    print(response.content)

{
  "claims": {
    "birthdate": "01/01/2000",
    "firstname": "Alice",
    "lastname": "Wonderland"
  },
  "createdAt": "2022-12-14T18:42:13Z",
  "protocolState": "RequestPending",
  "recordId": "08eb1750-a446-4b06-b89f-34c8cea31e64",
  "role": "Holder",
  "subjectId": "did:peer:2.Ez6LSdEYSAtkCArFgUXXY5g4RhXP23ej1Ynu8UejzjU9YrX3E.Vz6Mksj7wesBWcbKV7so6ipvaKYY9wse6aXc7rEYxWLXdJ8zw.SeyJ0IjoiZG0iLCJzIjoiaHR0cDovL2hvc3QuZG9ja2VyLmludGVybmFsOjgwOTAvZGlkY29tbSIsInIiOltdLCJhIjpbImRpZGNvbW0vdjIiXX0",
  "updatedAt": "2022-12-14T18:42:19Z"
}


In [72]:
path = '/prism-agent/issue-credentials/records'

# Make the request
response = requests.get(base_url + path, headers=headers)

# Check the response status code
if response.status_code == 200:
    # Print the response content
    loaded_json = json.loads(response.content.decode())
    print(json.dumps(loaded_json, indent=2))
    
else:
    # Print an error message
    print("Error: The API returned a non-200 status code")
    print(response.content)

{
  "count": 1,
  "items": [
    {
      "claims": {
        "birthdate": "01/01/2000",
        "firstname": "Alice",
        "lastname": "Wonderland"
      },
      "createdAt": "2022-12-14T18:42:13Z",
      "jwtCredential": "ImV5SjBlWEFpT2lKS1YxUWlMQ0poYkdjaU9pSkZVekkxTmlKOS5leUpwYzNNaU9pSmthV1E2Y0hKcGMyMDZZekF4T0dJMU1HUXRZalpsWXkwMFlUSXlMV0l4TXpVdE5qVmtNbVZsTWpCaU16RTFJaXdpYzNWaUlqb2laR2xrT25CbFpYSTZNaTVGZWpaTVUyUkZXVk5CZEd0RFFYSkdaMVZZV0ZrMVp6UlNhRmhRTWpObGFqRlpiblU0VldWcWVtcFZPVmx5V0RORkxsWjZOazFyYzJvM2QyVnpRbGRqWWt0V04zTnZObWx3ZG1GTFdWazVkM05sTm1GWVl6ZHlSVmw0VjB4WVpFbzRlbmN1VTJWNVNqQkphbTlwV2tjd2FVeERTbnBKYW05cFlVaFNNR05FYjNaTU1taDJZek5SZFZwSE9XcGhNbFo1VEcxc2RXUkhWbmxpYlVaelQycG5kMDlVUVhaYVIyeHJXVEk1ZEdKVFNYTkpia2xwVDJ4MFpFeERTbWhKYW5CaVNXMVNjRnBIVG5aaVZ6QjJaR3BKYVZoWU1DSXNJblpqSWpwN0ltTnlaV1JsYm5ScFlXeFRkV0pxWldOMElqcDdJbUpwY25Sb1pHRjBaU0k2SWpBeEx6QXhMekl3TURBaUxDSm1hWEp6ZEc1aGJXVWlPaUpCYkdsalpTSXNJbXhoYzNSdVlXMWxJam9pVjI5dVpHVnliR0Z1WkNJc0ltbGtJam9pWkdsa09uQmxaWEk2TWk1RmVqWk1VM

### Decode JWT in Credential

In [73]:
token = loaded_json['items'][0]['jwtCredential']
# print(token)

padded = token + "="*divmod(len(token),4)[1]
# print(padded)
jsondata = base64.urlsafe_b64decode(token)
# print(jsondata)
b64decoded_token = json.loads(jsondata.decode())
print(b64decoded_token,'\n')
header = jwt.get_unverified_header(b64decoded_token)
print (header,'\n')
# print (header['typ'])
# print (header['alg'])


# decodedJWT = jwt.decode(token, algorithms=[header['alg'], ])
# try:
#     jwt_decoded_id_token = jwt.decode(
#         token,
#         jwt_public_key,
#         # audience = config_idporten["aud"],
#         algorithms=[header['alg'], ])
#     print(jwt_decoded_id_token)
# except (jwt.ExpiredSignatureError, jwt.InvalidAudienceError) as e:
#     print("[ERROR]", e)

try:
    jwt_decoded_id_token = jwt.decode(b64decoded_token, options={"verify_signature": False})
    # print(jwt_decoded_id_token)
    print(json.dumps(jwt_decoded_id_token, indent=2))
except (jwt.ExpiredSignatureError, jwt.InvalidAudienceError) as e:
    print("[ERROR]", e)
    

eyJ0eXAiOiJKV1QiLCJhbGciOiJFUzI1NiJ9.eyJpc3MiOiJkaWQ6cHJpc206YzAxOGI1MGQtYjZlYy00YTIyLWIxMzUtNjVkMmVlMjBiMzE1Iiwic3ViIjoiZGlkOnBlZXI6Mi5FejZMU2RFWVNBdGtDQXJGZ1VYWFk1ZzRSaFhQMjNlajFZbnU4VWVqempVOVlyWDNFLlZ6Nk1rc2o3d2VzQldjYktWN3NvNmlwdmFLWVk5d3NlNmFYYzdyRVl4V0xYZEo4encuU2V5SjBJam9pWkcwaUxDSnpJam9pYUhSMGNEb3ZMMmh2YzNRdVpHOWphMlZ5TG1sdWRHVnlibUZzT2pnd09UQXZaR2xrWTI5dGJTSXNJbklpT2x0ZExDSmhJanBiSW1ScFpHTnZiVzB2ZGpJaVhYMCIsInZjIjp7ImNyZWRlbnRpYWxTdWJqZWN0Ijp7ImJpcnRoZGF0ZSI6IjAxLzAxLzIwMDAiLCJmaXJzdG5hbWUiOiJBbGljZSIsImxhc3RuYW1lIjoiV29uZGVybGFuZCIsImlkIjoiZGlkOnBlZXI6Mi5FejZMU2RFWVNBdGtDQXJGZ1VYWFk1ZzRSaFhQMjNlajFZbnU4VWVqempVOVlyWDNFLlZ6Nk1rc2o3d2VzQldjYktWN3NvNmlwdmFLWVk5d3NlNmFYYzdyRVl4V0xYZEo4encuU2V5SjBJam9pWkcwaUxDSnpJam9pYUhSMGNEb3ZMMmh2YzNRdVpHOWphMlZ5TG1sdWRHVnlibUZzT2pnd09UQXZaR2xrWTI5dGJTSXNJbklpT2x0ZExDSmhJanBiSW1ScFpHTnZiVzB2ZGpJaVhYMCJ9fSwibmJmIjoiMjAyMi0xMi0xNFQxODo0MjozNS43MjAzMTdaIiwiZXhwIjoiMjAyMi0xMi0xNFQxOTo0MjozNS43MjAzMTdaIn0.Z1dpBR70__0-QTxAegmosQa0iaqCGvNS1m3YvlirVRg

## Create connection with verifier

## Head over to Verifier Notebook to crate connection invitation

## Fetch invitationUrl from Verifier Agent

In [74]:
%store -r invitationUrlbase64verifier
print(invitationUrlbase64verifier)

eyJpZCI6ImQ5MmE0NmUzLWUzNGItNGY4NS04M2UwLWM5MGU5ZjVlNjcwMyIsInR5cGUiOiJodHRwczovL2RpZGNvbW0ub3JnL291dC1vZi1iYW5kLzIuMC9pbnZpdGF0aW9uIiwiZnJvbSI6ImRpZDpwZWVyOjIuRXo2TFNvV210WGpGWGNhUUVjalJ6djdGMWFMQWRiUWpqblh6eEhudTcyZmNodkNtNS5WejZNa3ZEaG1iWWlOVXJXdmJTdDVxYlVqZEZac0xES2dCVWJRS0ZTVll3cXJ6RUxXLlNleUowSWpvaVpHMGlMQ0p6SWpvaWFIUjBjRG92TDJodmMzUXVaRzlqYTJWeUxtbHVkR1Z5Ym1Gc09qa3dNREF2Wkdsa1kyOXRiU0lzSW5JaU9sdGRMQ0poSWpwYkltUnBaR052YlcwdmRqSWlYWDAiLCJib2R5Ijp7ImdvYWxfY29kZSI6ImNvbm5lY3QiLCJnb2FsIjoiRXN0YWJsaXNoIGEgdHJ1c3QgY29ubmVjdGlvbiBiZXR3ZWVuIHR3byBwZWVycyIsImFjY2VwdCI6W119fQ==


## Accept an OOB connection invitation from Verifier Agent
> Replace `{RAW_INVITATION}` with the value of the '_oob' query string parameter from the invitation URL above
```bash
curl -X 'POST' \
	'http://localhost:8090/prism-agent/connection-invitations' \
	-H 'Content-Type: application/json' \
	-d '{
		"invitation": "{RAW_INVITATION}"
	}' | jq
```

In [75]:
path = '/prism-agent/connection-invitations'
data = {"invitation": invitationUrlbase64verifier}
# Make the request
response = requests.post(base_url + path, headers=headers, json=data)

# Check the response status code
if response.status_code == 200:
    # Print the response content
    loaded_json = json.loads(response.content.decode())
    print(json.dumps(loaded_json, indent=2))
    invitation = json.loads(response.content.decode())
    invitationUrl = invitation['invitation']['invitationUrl']
    # invitationUrlbase64 = invitation['invitation']['invitationUrl'].split('=')[1]
    connection_id = invitation['connectionId']
else:
    # Print an error message
    print("Error: The API returned a non-200 status code")

{
  "connectionId": "aa7aaf4d-eeb8-437e-8318-19b0033d15fb",
  "createdAt": "2022-12-14T18:44:27Z",
  "invitation": {
    "from": "did:peer:2.Ez6LSoWmtXjFXcaQEcjRzv7F1aLAdbQjjnXzxHnu72fchvCm5.Vz6MkvDhmbYiNUrWvbSt5qbUjdFZsLDKgBUbQKFSVYwqrzELW.SeyJ0IjoiZG0iLCJzIjoiaHR0cDovL2hvc3QuZG9ja2VyLmludGVybmFsOjkwMDAvZGlkY29tbSIsInIiOltdLCJhIjpbImRpZGNvbW0vdjIiXX0",
    "id": "d92a46e3-e34b-4f85-83e0-c90e9f5e6703",
    "invitationUrl": "https://domain.com/path?_oob=eyJpZCI6ImQ5MmE0NmUzLWUzNGItNGY4NS04M2UwLWM5MGU5ZjVlNjcwMyIsInR5cGUiOiJodHRwczovL2RpZGNvbW0ub3JnL291dC1vZi1iYW5kLzIuMC9pbnZpdGF0aW9uIiwiZnJvbSI6ImRpZDpwZWVyOjIuRXo2TFNvV210WGpGWGNhUUVjalJ6djdGMWFMQWRiUWpqblh6eEhudTcyZmNodkNtNS5WejZNa3ZEaG1iWWlOVXJXdmJTdDVxYlVqZEZac0xES2dCVWJRS0ZTVll3cXJ6RUxXLlNleUowSWpvaVpHMGlMQ0p6SWpvaWFIUjBjRG92TDJodmMzUXVaRzlqYTJWeUxtbHVkR1Z5Ym1Gc09qa3dNREF2Wkdsa1kyOXRiU0lzSW5JaU9sdGRMQ0poSWpwYkltUnBaR052YlcwdmRqSWlYWDAiLCJib2R5Ijp7ImdvYWxfY29kZSI6ImNvbm5lY3QiLCJnb2FsIjoiRXN0YWJsaXNoIGEgdHJ1c3QgY29ubmVjdGlvbiBiZXR3ZWV

## Retrieving the list of connections
```bash
curl -X 'GET' 'http://localhost:8090/prism-agent/connections' | jq
```

In [78]:
import time
from termcolor import colored,cprint

path = '/prism-agent//connections/' + connection_id
# Make the request
response = requests.get(base_url + path, headers=headers)

# Check the response status code
if response.status_code == 200:
    # Print the response content
    loaded_json = json.loads(response.content.decode())
    print(json.dumps(loaded_json, indent=2))
    STATE = loaded_json['state']
else:
    # Print an error message
    print("Error: The API returned a non-200 status code")

# print('Current state for ConnectionId {} is {}'.format(connection_id,STATE))
print(colored("Current state for ConnectionId {} is {}".format(connection_id,STATE), "magenta", attrs=["bold"]))
while STATE != 'ConnectionResponseReceived':
    # Make the request
    response = requests.get(base_url + path, headers=headers)

    # Check the response status code
    if response.status_code == 200:
        # Print the response content
        loaded_json = json.loads(response.content.decode())
        # print(json.dumps(loaded_json, indent=2))
        STATE = loaded_json['state']
        print(colored("ConnectionId {0} is not in active state yet. State: {1}".format(connection_id,STATE), "yellow", attrs=["bold"]))
        their_did = loaded_json['theirDid']
    else:
        # Print an error message
        print("Error: The API returned a non-200 status code")
        

    time.sleep(1)
    
# print('ConnectionId: {0} is now active. Continue with notebook'.format(connection_id))
print(colored("ConnectionId: {0} is now active. Continue with notebook".format(connection_id), "green", attrs=["bold"]))


{
  "connectionId": "aa7aaf4d-eeb8-437e-8318-19b0033d15fb",
  "createdAt": "2022-12-14T18:44:27Z",
  "invitation": {
    "from": "did:peer:2.Ez6LSoWmtXjFXcaQEcjRzv7F1aLAdbQjjnXzxHnu72fchvCm5.Vz6MkvDhmbYiNUrWvbSt5qbUjdFZsLDKgBUbQKFSVYwqrzELW.SeyJ0IjoiZG0iLCJzIjoiaHR0cDovL2hvc3QuZG9ja2VyLmludGVybmFsOjkwMDAvZGlkY29tbSIsInIiOltdLCJhIjpbImRpZGNvbW0vdjIiXX0",
    "id": "d92a46e3-e34b-4f85-83e0-c90e9f5e6703",
    "invitationUrl": "https://domain.com/path?_oob=eyJpZCI6ImQ5MmE0NmUzLWUzNGItNGY4NS04M2UwLWM5MGU5ZjVlNjcwMyIsInR5cGUiOiJodHRwczovL2RpZGNvbW0ub3JnL291dC1vZi1iYW5kLzIuMC9pbnZpdGF0aW9uIiwiZnJvbSI6ImRpZDpwZWVyOjIuRXo2TFNvV210WGpGWGNhUUVjalJ6djdGMWFMQWRiUWpqblh6eEhudTcyZmNodkNtNS5WejZNa3ZEaG1iWWlOVXJXdmJTdDVxYlVqZEZac0xES2dCVWJRS0ZTVll3cXJ6RUxXLlNleUowSWpvaVpHMGlMQ0p6SWpvaWFIUjBjRG92TDJodmMzUXVaRzlqYTJWeUxtbHVkR1Z5Ym1Gc09qa3dNREF2Wkdsa1kyOXRiU0lzSW5JaU9sdGRMQ0poSWpwYkltUnBaR052YlcwdmRqSWlYWDAiLCJib2R5Ijp7ImdvYWxfY29kZSI6ImNvbm5lY3QiLCJnb2FsIjoiRXN0YWJsaXNoIGEgdHJ1c3QgY29ubmVjdGlvbiBiZXR3ZWV

## Verify Credentials

### Retrieving the list of presentation records

```shell
curl -X 'GET' 'http://localhost:8090/prism-agent/present-proof/presentations' -H 'accept: application/json' | jq
```

In [84]:
path = '/prism-agent/present-proof/presentations'

# Make the request
response = requests.get(base_url + path, headers=headers)

# Check the response status code
if response.status_code == 200:
    # Print the response content
    loaded_json = json.loads(response.content.decode())
    print(json.dumps(loaded_json, indent=2))
    presentationId = loaded_json[0]['presentationId']
else:
    # Print an error message
    print("Error: The API returned a non-200 status code")

[
  {
    "presentationId": "547d5d1c-01ca-46a4-8246-1fe81626a3ba",
    "proofs": [],
    "status": "RequestReceived"
  }
]


### Retrieving the list of credentials records
> choose the `{RECORD_ID}` for credential with status CredentialRecieved

```shell
curl -X 'GET' 'http://localhost:8090/prism-agent/issue-credentials/records' -H 'accept: application/json' | jq
```

In [85]:
print(record_id)
print(presentationId)

08eb1750-a446-4b06-b89f-34c8cea31e64
547d5d1c-01ca-46a4-8246-1fe81626a3ba


### Accepting the Presentation Request 
>Replace `{PRESENTATION_ID}` with the UUID of the record from the presentation records list

>Replace `{RECORD_ID}` with the UUID of the record from the credential records list


```shell
curl -X 'PATCH' \
  'http://localhost:8090/prism-agent/present-proof/presentations/{PRESENTATION_ID}' \
  -H 'Content-Type: application/json' \
  -d '{
  "action": "request-accept",
  "proofId": ["{RECORD_ID}"]
}'
```

In [91]:
data = {
  "action": "request-accept",
  "proofId": [record_id]
}

path = '/prism-agent/present-proof/presentations/' + presentationId

# Make the request
response = requests.patch(base_url + path, headers=headers, json=data)

# Check the response status code
if response.status_code == 200:
    # Print the response content
    print(response.content)
    # loaded_json = json.loads(response.content.decode())
    # print(json.dumps(loaded_json, indent=2))
    # proof_requestId = loaded_json[0]['presentationId']
else:
    # Print an error message
    print("Error: The API returned a non-200 status code")
    print(response.content)

b'Update the presentation record'


# TODO
> Fix response for successful API in PATCH HTTP call above

### check Presentation state  PresentationSent 
> check PresentationSent !
```shell
curl -X 'GET' 'http://localhost:8090/prism-agent/present-proof/presentations' -H 'accept: application/json' | jq
```

In [95]:
path = '/prism-agent/present-proof/presentations'

# Make the request
response = requests.get(base_url + path, headers=headers)

# Check the response status code
if response.status_code == 200:
    # Print the response content
    # print(response.content)
    # loaded_json = json.loads(response.content.decode())
    print(json.dumps(loaded_json, indent=2))
    presentationId = loaded_json[0]['presentationId']
else:
    # Print an error message
    print("Error: The API returned a non-200 status code")

[
  {
    "presentationId": "547d5d1c-01ca-46a4-8246-1fe81626a3ba",
    "proofs": [],
    "status": "RequestReceived"
  }
]
